In [1]:
import numpy as np

In [2]:
def InitialiseParticles(nParticles,xMin,xMax,nVariables,alpha,deltaT):
    
    r1 = np.random.uniform(size=(nParticles,nVariables))
    r2 = np.random.uniform(size=(nParticles,nVariables))
    
    positions = xMin + r1 * (xMax - xMin)
    velocities = alpha/deltaT * ((xMin - xMax)/2 + r2 * (xMax-xMin))
    
    return (positions, velocities)

In [11]:
def UpdateParticles(positions, velocities, particleBestPositions, 
                    swarmBestPositions, inertiaWeight, vMax, c1, c2, deltaT):
    
    q = np.random.uniform(size = (np.shape(positions)[0], 1))
    q = np.concatenate((q, q), axis = 1)
    r = np.random.uniform(size = (np.shape(positions)[0], 1))
    r = np.concatenate((r, r), axis = 1)
    
    particleBestDifference = particleBestPositions - positions
    swarmBestDifference = (positions - swarmBestPosition) * (-1)
    
    velocities = inertiaWeight*velocities + c1*q*particleBestDifference/deltaT + c2*r*swarmBestDifference/deltaT
    
    # now limit velocities to vMax
    
    absVelocities = np.sqrt(np.sum(np.power(velocities, 2), axis=1))
    tooFastParticleIndeces = (absVelocities > vMax)
    
    notEmpty = np.any(tooFastParticleIndeces) # checks if at least one element is true
    
    if notEmpty:
        indices = np.argwhere(tooFastParticleIndeces)
        for index in indices:
            velocities[index,:] = velocities[index,:] * vMax / absVelocities[index]
            
    newPositions = positions + velocities * deltaT
    
    return (newPositions, velocities)

In [5]:
def EvaluateParticles(positions):
    
    x1 = positions[:, 0]
    x2 = positions[:, 1]
    
    funcValues = 1 + np.power((-13 + x1 - np.power(x2, 3) + 5 * np.power(x2, 2) - 2*x2), 2) + \
        np.power((-29 + x1 + np.power(x2, 3) + np.power(x2, 2) - 14 * x2), 2)
        
    return funcValues

In [13]:
nIterations = 1000;
nParticles = 40;
nVariables = 2;
xMin = -10;
xMax = 10;
alpha = 1; 
deltaT = 1; 
vMax = (xMax-xMin) / deltaT;
c1 = 2; 
c2 = 2; 
inertiaWeight = 1.4;
inertiaWeightMin = 0.3;
inertiaWeightReduction = 0.99;

swarmBest = 1e20
swarmBestPositions = np.zeros((1,nVariables))
particleBest = 1e20 * np.ones((nParticles, 1))
particleBestPositions = np.zeros((nParticles, nVariables))

(positions, velocities) = InitialiseParticles(nParticles, xMin,xMax,nVariables,alpha,deltaT)

for iteration in range(nIterations):
    
    funcValues = EvaluateParticles(positions)
    
    for iParticle in range(nParticles):
        
        isSwarmBest = (funcValues[iParticle] < swarmBest)
        isParticleBest = (funcValues[iParticle] < particleBest[iParticle])
        
        if isSwarmBest:
            swarmBestPosition = positions[iParticle,:]
            swarmBest = funcValues[iParticle]
            particleBestPositions[iParticle,:] = positions[iParticle,:]
            print('Iteration %d, swarm best: %.16f' % (iteration, swarmBest))
        elif isParticleBest:
            particleBestPositions[iParticle,:] = positions[iParticle,:]
            
    (positions, velocities) = UpdateParticles(positions,velocities,
        particleBestPositions,swarmBestPosition,inertiaWeight,vMax,c1,c2,deltaT)
    
    isReduce = (inertiaWeight > inertiaWeightMin)
    if isReduce:
        inertiaWeight = inertiaWeight * inertiaWeightReduction

print('Lowest value after %d iterations is %.16f,' % (nIterations,swarmBest))
print('this occurrs at coordinates (x,y) = (%.3f,%.3f)' % (swarmBestPosition[0], swarmBestPosition[1]))

Iteration 0, swarm best: 65920.1149503990745870
Iteration 0, swarm best: 25564.5858733641907747
Iteration 0, swarm best: 12289.3640187986147794
Iteration 0, swarm best: 1303.9581962931006274
Iteration 0, swarm best: 92.0761371101759778
Iteration 0, swarm best: 73.0926380339688535
Iteration 3, swarm best: 58.4938796880636005
Iteration 13, swarm best: 52.0580849890383917
Iteration 14, swarm best: 17.6378331415352640
Iteration 46, swarm best: 13.9907784882350494
Iteration 47, swarm best: 7.2136129181571533
Iteration 53, swarm best: 6.1784414308310049
Iteration 55, swarm best: 5.6237627677976993
Iteration 66, swarm best: 3.1414313323532257
Iteration 71, swarm best: 2.3359417808185947
Iteration 81, swarm best: 2.2632260610612369
Iteration 90, swarm best: 2.2562379015697527
Iteration 91, swarm best: 2.2541872405847201
Iteration 92, swarm best: 2.2381549885918512
Iteration 92, swarm best: 2.2331718067793340
Iteration 92, swarm best: 2.1825368931300346
Iteration 93, swarm best: 2.1415150106098